##Analysis of NN based activation functions: y=g(x)

### Preparation
Start your own Colab runtime with this command:

```
rabbit --verifiable build -c opt \
  --define=babelfish_task=mt \
  //learning/brain/research/babelfish/colab:colab_notebook.par

borgcfg learning/brain/research/babelfish/colab/colab.borg \
  --vars="task=mt,cell=vz,charged_user=translate-train" \
  --skip_confirmation reload
```
Then connect your colab against this runtime
(it will be called 'BabelFish Colab USERNAME (mt)').

In [ ]:
# load the model from the checkpoint 
ml_dash_name = 'esn_actNN_gx_rnnL2_norm1_run08'
checkpoint = '/cns/vz-d/home/harshx/brain/rs=6.3/'+ ml_dash_name +'/train/' + 'ckpt-00020600' 

In [ ]:
from google3.pyglib import build_data, resources
print(build_data.BuildData())

from colabtools import adhoc_import
with adhoc_import.Google3SubmittedChangelist(
    behavior='preferred',
    build_targets=['//third_party/py/networkx:utils_tests_test_misc.python3']):
  adhoc_import.Reload(resources)
  import networkx as nx

help(nx)

Built on Thu May 21 21:57:21 2020 (1590123441)
Built by harshx@odqb3.prod.google.com:/google/src/cloud/harshx/82/.snapshot/391/google3
Built as //learning/brain/research/babelfish/colab:colab_notebook
Build ID: 1dc474dc-49c8-4b3b-84a2-97b7c37dacad
Built from changelist 312358057 in a modified client based on //depot/google3 in CitC workspace build-secure-info:source-uri at snapshot 391
Build platform: gcc-4.X.Y-crosstool-v18-llvm-grtev4-k8
Build tool: Blaze, release blaze-2020.05.10-1 (mainline @310756764)
Built with par options [--compress]
Currently running under Python 3.6.7: embedded.



Help on package networkx:

NAME
    networkx

DESCRIPTION
    NetworkX
    
    NetworkX is a Python package for the creation, manipulation,
    and study of the structure, dynamics, and functions
    of complex networks.
    
    Website (including documentation)::
    
        http://networkx.github.io
    
    Mailing list::
    
        https://groups.google.com/forum/#!forum/networkx-discuss
    
    Source::
    
        https://github.com/networkx/networkx
    
    Bug reports::
    
        https://github.com/networkx/networkx/issues
    
    Simple example
    --------------
    
    Find the shortest path between two nodes in an undirected graph::
    
        >>> import networkx as nx
        >>> G = nx.Graph()
        >>> G.add_edge('A', 'B', weight=4)
        >>> G.add_edge('B', 'D', weight=2)
        >>> G.add_edge('A', 'C', weight=3)
        >>> G.add_edge('C', 'D', weight=4)
        >>> nx.shortest_path(G, 'A', 'D', weight='weight')
        ['A', 'B', 'D']
    
    Bugs

#Checking the weight topology code
1. SCR = lower subdiag + upper right corner
2. Chain
3. Permutation
4. Ring
5. Random

Note: this is the mask which will be multiplied to the inititialized matrix

In [ ]:
from __future__ import print_function
import tensorflow as tf
import sys, math
import numpy as np
print(tf.__version__)

2.2.0


In [ ]:
w_random = tf.random.uniform([3, 3])
tensor = tf.range(10)
tf.print(w_random)
tf.print(tensor, output_stream=sys.stderr)
# print(w_random)

<tf.Operation 'PrintV2_14' type=PrintV2>

In [ ]:
  sess = tf.compat.v1.Session()  
  with sess.as_default():  
      w_random = tf.random.uniform([3, 3]) 
      print_op = tf.print("tensors:", w_random,  
                          output_stream=sys.stdout)  
      # with tf.control_dependencies([print_op]):  
      #  tripled_tensor = tensor * 3  
      sess.run(tripled_tensor) 

tensors: [[0.0363430977 0.568793416 0.900764346]
 [0.681818485 0.701964736 0.88309145]
 [0.366456389 0.958831191 0.102034092]]


In [ ]:
tensor = tf.range(10)  
tf.print(tensor, output_stream=sys.stdout)

<tf.Operation 'PrintV2_18' type=PrintV2>

In [ ]:
# scr_lower (sub-diag)
sess2 = tf.compat.v1.Session()  

with sess2.as_default(): # initialize the reservoir weights
      shape = (5, 5)
      diagonal = tf.ones(shape[0])  
      w_diag = tf.linalg.diag(diagonal)#, k = -1)
      w_scr = tf.roll(w_diag, shift=[1, 0], axis=[0, 1]) 
      print_op = tf.print("SCR-lower:\n", w_scr,  
                          output_stream=sys.stdout)  
      with tf.control_dependencies([print_op]):  
        tripled_tensor = w_scr * 3  
      sess2.run(tripled_tensor) 

SCR-lower:
 [[0 0 0 0 1]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]]


In [ ]:
# scr_upper 
sessu = tf.compat.v1.Session()  

with sessu.as_default(): # initialize the reservoir weights
      shape = (5, 5)
      diagonal = tf.ones(shape[0])  
      w_diag = tf.linalg.diag(diagonal)#, k = -1)
      w_scr = tf.roll(w_diag, shift=[-1, 0], axis=[0, 1]) 
      print_op = tf.print("SCR-upper:\n", w_scr,  
                          output_stream=sys.stdout)  
      with tf.control_dependencies([print_op]):  
        tripled_tensor = w_scr * 3  
      sessu.run(tripled_tensor) 

SCR-upper:
 [[0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]
 [1 0 0 0 0]]


In [ ]:
# scr_both
sessb = tf.compat.v1.Session()  

with sessb.as_default(): # initialize the reservoir weights
      shape = (5, 5)
      diagonal = tf.ones(shape[0])  
      w_diag = tf.linalg.diag(diagonal)#, k = -1)
      w_scr = tf.roll(w_diag, shift=[-1, 0], axis=[0, 1]) + tf.roll(w_diag, shift=[1, 0], axis=[0, 1]) 
      print_op = tf.print("tensors:", w_scr,  
                          output_stream=sys.stdout)  
      with tf.control_dependencies([print_op]):  
        tripled_tensor = w_scr * 3  
      sessb.run(tripled_tensor) 

tensors: [[0 1 0 0 1]
 [1 0 1 0 0]
 [0 1 0 1 0]
 [0 0 1 0 1]
 [1 0 0 1 0]]


In [ ]:
# scr_bd (both+diag)
sessbd = tf.compat.v1.Session()  

with sessbd.as_default(): # initialize the reservoir weights
      shape = (5, 5)
      diagonal = tf.ones(shape[0])  
      w_diag = tf.linalg.diag(diagonal)#, k = -1)
      w_scr = tf.roll(w_diag, shift=[-1, 0], axis=[0, 1]) + tf.roll(w_diag, shift=[1, 0], axis=[0, 1]) +w_diag
      print_op = tf.print("SCR-bd:\n", w_scr,  
                          output_stream=sys.stdout)  
      with tf.control_dependencies([print_op]):  
        tripled_tensor = w_scr * 3  
      sessbd.run(tripled_tensor) 

SCR-bd:
 [[1 1 0 0 1]
 [1 1 1 0 0]
 [0 1 1 1 0]
 [0 0 1 1 1]
 [1 0 0 1 1]]


In [ ]:
# for input weight: scrl topology
sessil = tf.compat.v1.Session()  

with sessil.as_default(): # initialize the reservoir weights
      shape = (3, 6)
      # shape0, shape1 = shape[-2], shape[-1]
      dim0, dim1 = shape[-2], shape[-1]
      if dim0 <= dim1:
        shape0, shape1 = dim0, dim1
        do_transpose = False
      else:
        shape0, shape1 = dim1, dim0
        do_transpose = True
      # assuming shape0 < shape1:
      repeat = math.ceil(shape1/shape0) - 1
      w_scr = tf.eye(shape0)
      for i in range(repeat):
        if i == repeat - 1: # truncate
          w_diag = tf.eye(shape0)
          w_scr = tf.concat([w_scr, w_diag[:, :(shape1-shape0)]], axis=1)
        else:
          w_scr = tf.concat([w_scr, tf.eye(shape0) ], axis=1)
      w_scr = tf.roll(w_scr, shift=[-1, 0], axis=[0, 1])
      # the transpose condition
      if do_transpose:
        w_scr = tf.transpose(w_scr, perm=[1, 0])

      print_op = tf.print("tensors:", w_scr,  
                          output_stream=sys.stdout)  
      with tf.control_dependencies([print_op]):  
        tripled_tensor = w_scr * 3  
      sessil.run(tripled_tensor) 

tensors: [[0 1 0 0 1 0]
 [0 0 1 0 0 1]
 [1 0 0 1 0 0]]


In [ ]:
# for input weight: scrbd topology
sessibd = tf.compat.v1.Session()  

with sessibd.as_default(): # initialize the reservoir weights
      shape = (6,6)
      # shape0, shape1 = shape[-2], shape[-1]
      dim0, dim1 = shape[-2], shape[-1]
      if dim0 <= dim1:
        shape0, shape1 = dim0, dim1
        do_transpose = False
      else:
        shape0, shape1 = dim1, dim0
        do_transpose = True
      # assuming shape0 < shape1:
      repeat = math.ceil(shape1/shape0) - 1
      w_scr = tf.eye(shape0)
      for i in range(repeat):
        if i == repeat - 1: # truncate
          w_diag = tf.eye(shape0)
          w_scr = tf.concat([w_scr, w_diag[:, :(shape1-shape0)]], axis=1)
        else:
          w_scr = tf.concat([w_scr, tf.eye(shape0) ], axis=1)
      D_mat = w_scr
      adj_mat = tf.roll(w_scr, shift=[-1, 0], axis=[0, 1]) + tf.roll(w_scr, shift=[1, 0], axis=[0, 1])
      w_scr = adj_mat + D_mat
      # the transpose condition
      if do_transpose:
        w_scr = tf.transpose(w_scr, perm=[1, 0])

      mask = w_scr # tf.cast(w_scr, dtype=dtype)
      # should have v and mask
      # D_mat = D_mat*tf.reduce_sum(adj_mat, axis=-1)[0]  # scalar
      # v = D_mat - adj_mat
      print_op = tf.print("tensors:", mask,  
                          output_stream=sys.stdout)  
      with tf.control_dependencies([print_op]):  
        tripled_tensor = w_scr * 3  
      sessibd.run(tripled_tensor) 

tensors: [[1 1 0 0 0 1]
 [1 1 1 0 0 0]
 [0 1 1 1 0 0]
 [0 0 1 1 1 0]
 [0 0 0 1 1 1]
 [1 0 0 0 1 1]]


In [ ]:
# rotation matrix which takes care of the average sequence length
# scr_lower (sub-diag)
sessr = tf.compat.v1.Session()  

with sessr.as_default(): # initialize the reservoir weights
      shape = (6, 6)
      avg_L = 3 # average sequence length
      num_rotates = math.ceil(shape[-1]/avg_L) # NOTE: take the smaller shape, as we already repeat for the longer dim.
      print(num_rotates)
      diagonal = tf.ones(shape[0])  
      w_diag = tf.linalg.diag(diagonal)#, k = -1)
      w_scr_base = tf.roll(w_diag, shift=[1, 0], axis=[0, 1])
      mat = []
      
      for r in range(num_rotates):
        temp = tf.roll(w_scr_base, shift=[r*avg_L, 0], axis=[0, 1])
        if r==0:
          w_scr = temp
        else:
          w_scr = w_scr + temp
        mat.append(temp)
      print_op = tf.print("tensors:", w_scr,mat,  
                          output_stream=sys.stdout)  
      with tf.control_dependencies([print_op]):  
        tripled_tensor = w_scr * 3  
      sessr.run(tripled_tensor) 

In [ ]:
# Reservoir rotation matrix : which takes care of the average sequence length

sessr = tf.compat.v1.Session()  

with sessr.as_default(): # initialize the reservoir weights
      shape = (6, 6)
      diagonal = tf.ones(shape[0])  
      w_diag = tf.linalg.diag(diagonal)#, k = -1)
      w_scr = tf.roll(w_diag, shift=[1, 0], axis=[0, 1])

      avg_L = 3 # average sequence length
      num_rotates = math.ceil(shape[-1]/avg_L)
      for r in range(num_rotates):
        temp = tf.roll(w_scr, shift=[r*avg_L, 0], axis=[0, 1])
        if r==0:
          w_rot = temp
        else:
          w_rot = w_rot + temp
      print_op = tf.print("Rotation:\n", w_rot,
                          output_stream=sys.stdout)  
      with tf.control_dependencies([print_op]):  
        tripled_tensor = w_scr * 3  
      sessr.run(tripled_tensor) 

Rotation:
 [[0 0 1 0 0 1]
 [1 0 0 1 0 0]
 [0 1 0 0 1 0]
 [0 0 1 0 0 1]
 [1 0 0 1 0 0]
 [0 1 0 0 1 0]]


In [ ]:
# for input weight: rotation topology
sessir = tf.compat.v1.Session()  

with sessir.as_default(): # initialize the reservoir weights
      shape = (6,6)
      # shape0, shape1 = shape[-2], shape[-1]
      dim0, dim1 = shape[-2], shape[-1]
      if dim0 <= dim1: # shape0 is the smaller dim
        shape0, shape1 = dim0, dim1
        do_transpose = False
      else:
        shape0, shape1 = dim1, dim0
        do_transpose = True
      # assuming shape0 < shape1:
      repeat = math.ceil(shape1/shape0) - 1
      w_scr = tf.eye(shape0)
      for i in range(repeat):
        if i == repeat - 1: # truncate
          w_diag = tf.eye(shape0)
          w_scr = tf.concat([w_scr, w_diag[:, :(shape1-shape0)]], axis=1)
        else:
          w_scr = tf.concat([w_scr, tf.eye(shape0) ], axis=1)
      w_scr = tf.roll(w_scr, shift=[1, 0], axis=[0, 1])
      # the transpose condition
      if do_transpose:
        w_scr = tf.transpose(w_scr, perm=[1, 0])
      
      avg_L = 3 # average sequence length
      num_rotates = math.ceil(shape0/avg_L)
      for r in range(num_rotates):
        temp = tf.roll(w_scr, shift=[r*avg_L, 0], axis=[0, 1])
        if r==0:
          w_rot = temp
        else:
          w_rot = w_rot + temp

      print_op = tf.print("tensors:", w_rot,  w_rot[:6, :6],
                          output_stream=sys.stdout)  
      with tf.control_dependencies([print_op]):  
        tripled_tensor = w_scr * 3  
      sessir.run(tripled_tensor) 

tensors: [[0 0 1 0 0 1]
 [1 0 0 1 0 0]
 [0 1 0 0 1 0]
 [0 0 1 0 0 1]
 [1 0 0 1 0 0]
 [0 1 0 0 1 0]] [[0 0 1 0 0 1]
 [1 0 0 1 0 0]
 [0 1 0 0 1 0]
 [0 0 1 0 0 1]
 [1 0 0 1 0 0]
 [0 1 0 0 1 0]]


In [ ]:

sessa = tf.compat.v1.Session()  

with sessa.as_default():
      shape = (5, 6)
      mat = tf.random.normal(shape)  
      #w_diag = tf.linalg.diag(diagonal)#, k = -1)
      # w_scr = mat[mat<0] #tf.roll(mat, shift=[1, 0], axis=[0, 1]) 
      # w_scr = tf.maximum(mat, 0)
      #mask_np = tf.make_ndarray(mat)
      #print(mask_np)
      w_scr_mask = tf.where(mat>0, tf.ones(mat.shape), tf.zeros(mat.shape))
      w_scr = mat*w_scr_mask
      print_op = tf.print("tensors:", w_scr, 'mat', mat, w_scr_mask,  
                          output_stream=sys.stdout)  
      with tf.control_dependencies([print_op]):  
        tripled_tensor = w_scr * 3  
      sessa.run(tripled_tensor) 

tensors: [[-0 -0 0.490858823 -0 0.402715683 -0]
 [-0 -0 -0 -0 0.704687059 0.204413876]
 [0.770711303 0.257065117 -0 -0 0.442749828 1.3006258]
 [0.0617309 -0 -0 1.62153125 -0 0.691740453]
 [-0 0.0859779418 0.742853224 0.87780422 -0 0.880521715]] mat [[-0.0185288079 -0.483362049 0.490858823 -0.414317876 0.402715683 -0.526476204]
 [-2.8118341 -2.16614079 -0.303864509 -0.528555155 0.704687059 0.204413876]
 [0.770711303 0.257065117 -0.0175612476 -0.305617481 0.442749828 1.3006258]
 [0.0617309 -0.0796477646 -1.0349772 1.62153125 -0.597598433 0.691740453]
 [-1.23412371 0.0859779418 0.742853224 0.87780422 -0.3288396 0.880521715]] [[0 0 1 0 1 0]
 [0 0 0 0 1 1]
 [1 1 0 0 1 1]
 [1 0 0 1 0 1]
 [0 1 1 1 0 1]]


In [ ]:
# convert list of tensors to tensor
# x = tf.constant([1, 4])
# y = tf.constant([2, 5])
# z = tf.constant([3, 6])
# tf.stack([x, y, z])
sess = tf.compat.v1.Session()
with sess.as_default():
  x1 = tf.zeros([7, 5])
  y1 = tf.ones([7, 5])

  # tf.print(x1)

  z1 = tf.concat([tf.reshape(x1, [*x1.shape, 1]), tf.reshape(y1, [*y1.shape,1])], axis=-1)
  #cz1 = tf.concat([x1, y1], axis=-1)
  print_op = tf.print("tensors:", x1.shape, y1.shape, z1.shape,
                          output_stream=sys.stdout)  
  with tf.control_dependencies([print_op]):  
    tripled_tensor =z1* 3  
  sess.run(tripled_tensor) 



tensors: TensorShape([Dimension(7), Dimension(5)]) TensorShape([Dimension(7), Dimension(5)]) TensorShape([Dimension(7), Dimension(5), Dimension(2)])


In [ ]:
['scrl']*10

['scrl',
 'scrl',
 'scrl',
 'scrl',
 'scrl',
 'scrl',
 'scrl',
 'scrl',
 'scrl',
 'scrl']

In [ ]:
def _old_projection_out(input_vec, in_wt_vector, rot_num, in_dim, res_dim):
  print('proj out :', input_vec, in_wt_vector, rot_num, input_vec.shape,
        in_wt_vector.shape)
  print('shape check : ', res_dim, in_dim)
  # case I:  res_dim >= in_dim
  if res_dim >= in_dim :
    # concat inputs to match the res_sim
    repeat = math.ceil(res_dim / in_dim) - 1 
    print('repeat: ', repeat)
    for i in range(repeat):
      if i == repeat - 1:
        input_vec = tf.concat([input_vec, input_vec[:, :(res_dim-repeat*(in_dim-1)-1)]], 
                              axis=-1)
      else:
        input_vec = tf.concat([input_vec, input_vec], axis=-1)
    # rotate the input_vec
    input_vec = tf.roll(input_vec, shift=[0, rot_num], axis=[-2, -1])
    print('final in vec: ', input_vec, in_wt_vector)
    proj_vec = in_wt_vector * input_vec
  else: # Case II: res_dim < in_dim
    repeat = math.ceil(in_dim/res_dim)-1
    print('repeat num :', repeat, res_dim, in_dim, input_vec.shape)
    # pad zeros
    zeros_vec = tf.zeros([input_vec.shape[0], in_dim-repeat*(res_dim-1)-1])
    print('zeros: ', zeros_vec)
    input_vec = tf.concat([input_vec, zeros_vec], axis=-1)
    print('check: ', input_vec) # BxH
    # reshape the input_vec
    # input_vec = tf.transpose(tf.expand_dims(input_vec, axis=-1)) #
    input_vec = tf.reshape(input_vec, [input_vec.shape[0], repeat+1, res_dim])
    input_vec = tf.transpose(input_vec, perm=[0, 2, 1])
    # input_vec = tf.squeeze(input_vec, axis=-1)
    print('check reshape: ', input_vec.shape) # B x H x r
    input_vec = tf.roll(input_vec, shift=[rot_num, 0], axis=[1, 2])
    print('after roll: ', input_vec, tf.transpose(in_wt_vector))
    proj_vec = tf.math.reduce_sum(input_vec*tf.transpose(in_wt_vector), axis=-1) # sum[(Hxr) * (Hxr)]
    #print(proj_vec)
    #proj_vec = input_vec

  # rotate the input embeddings and return the hadamard product
  return proj_vec  # [?, H]

In [ ]:
# Individual vectors: inefficient implementation of ESN basis idea
def _reservoir_out(hidden_vec, res_wt_vector, rot_num):
  print('res_out: ', hidden_vec, res_wt_vector, rot_num, hidden_vec.shape,
        res_wt_vector.shape)
  # rotate the hidden vector and return the hadamard product
  hidden_vec = tf.roll(hidden_vec, shift=[0, rot_num], axis=[0, 1])
  print('res out 2: ', res_wt_vector*hidden_vec)
  return res_wt_vector*hidden_vec  # [?, H]

def _projection_out(input_vec, in_wt_vector, rot_num, in_dim, res_dim):
  print('proj out :', input_vec, in_wt_vector, rot_num, input_vec.shape,
        in_wt_vector.shape)
  print('shape check : ', res_dim, in_dim)
  # case I:  res_dim >= in_dim
  if res_dim >= in_dim :
    # concat inputs to match the res_sim
    repeat = math.ceil(res_dim / in_dim) - 1 
    print('repeat: ', repeat)
    for i in range(repeat):
      if i == repeat - 1:
        input_vec = tf.concat([input_vec, input_vec[:, :(res_dim-repeat*(in_dim-1)-1)]], 
                              axis=-1)
      else:
        input_vec = tf.concat([input_vec, input_vec], axis=-1)
    # rotate the input_vec
    input_vec = tf.roll(input_vec, shift=[0, rot_num], axis=[-2, -1])
    print('final in vec: ', input_vec, in_wt_vector)
    proj_vec = in_wt_vector * input_vec
  else: # Case II: res_dim < in_dim
    repeat = math.ceil(in_dim/res_dim)-1
    print('repeat num :', repeat, res_dim, in_dim, input_vec.shape)
    # pad zeros
    zeros_vec = tf.zeros([input_vec.shape[0], in_dim-repeat*(res_dim-1)-1])
    # zeros_vec = tf.zeros([in_dim-repeat*(res_dim-1)-1])
    print('zeros: ', zeros_vec, input_vec)
    input_vec = tf.concat([input_vec, zeros_vec], axis=-1)
    print('check: ', input_vec) # BxH
    # reshape the input_vec
    # input_vec = tf.transpose(tf.expand_dims(input_vec, axis=-1)) #
    input_vec = tf.reshape(input_vec, [input_vec.shape[0], repeat+1, res_dim])
    input_vec = tf.transpose(input_vec, perm=[0, 2, 1])
    # input_vec = tf.squeeze(input_vec, axis=-1)
    print('check reshape: ', input_vec.shape) # B x H x r
    input_vec = tf.roll(input_vec, shift=[rot_num, 0], axis=[1, 2])
    print('after roll: ', input_vec, tf.transpose(in_wt_vector))
    proj_vec = tf.math.reduce_sum(input_vec*tf.transpose(in_wt_vector), axis=-1) # sum[(Hxr) * (Hxr)]
    #print(proj_vec)
    #proj_vec = input_vec

  # rotate the input embeddings and return the hadamard product
  return proj_vec  # [?, H]

sess = tf.compat.v1.Session()
with sess.as_default():
  res_dim, in_dim = 4, 6# 6, 4
  if res_dim >= in_dim:
    repeat_dim = 1
  else:
    print('res_dim < in_dim', in_dim/res_dim, math.ceil(in_dim/res_dim))
    repeat_dim = math.ceil(in_dim/res_dim)
  B = 3 # batch size
  hidden_vec = tf.random.normal([B, res_dim], seed=9)
  res_wt_vector = tf.ones([1, res_dim])*10
  
  roll_hidden_vec = tf.roll(hidden_vec, shift=[0, 2], axis=[0, 1])
  
  input_vec = tf.random.normal([B, in_dim], seed=9)
  in_wt_vector = tf.ones([repeat_dim, res_dim])*10

  # tf.print(x1)
  rot=1
  z1 = _reservoir_out(hidden_vec, res_wt_vector, rot)
  z2 = _projection_out(input_vec, in_wt_vector, rot, in_dim, res_dim)
  #cz1 = tf.concat([x1, y1], axis=-1)
  # print_op = tf.print("reservoir_out:", hidden_vec, roll_hidden_vec, res_wt_vector, z1, z1.shape,
  #                         output_stream=sys.stdout)  
  print_op = tf.print("projection_out:", input_vec, in_wt_vector, z2, z2.shape,
                          output_stream=sys.stdout)  
  with tf.control_dependencies([print_op]):  
    tripled_tensor =z1* 3  
  sess.run(tripled_tensor) 

res_dim < in_dim 1.5 2
res_out:  Tensor("random_normal_288:0", shape=(3, 4), dtype=float32) Tensor("mul_739:0", shape=(1, 4), dtype=float32) 1 (3, 4) (1, 4)
res out 2:  Tensor("mul_741:0", shape=(3, 4), dtype=float32)
proj out : Tensor("random_normal_289:0", shape=(3, 6), dtype=float32) Tensor("mul_740:0", shape=(2, 4), dtype=float32) 1 (3, 6) (2, 4)
shape check :  4 6
repeat num : 1 4 6 (3, 6)
zeros:  Tensor("zeros_115:0", shape=(3, 2), dtype=float32) Tensor("random_normal_289:0", shape=(3, 6), dtype=float32)
check:  Tensor("concat_121:0", shape=(3, 8), dtype=float32)
check reshape:  (3, 4, 2)
after roll:  Tensor("Roll_359:0", shape=(3, 4, 2), dtype=float32) Tensor("transpose_137:0", shape=(4, 2), dtype=float32)
projection_out: [[1.15507281 -0.491057307 0.444480509 -2.43835855 -1.99601936 -0.484241098]
 [-0.491944462 0.527831 1.04352212 3.02801514 -0.460363448 0.412836105]
 [-0.254990488 -0.215089872 0.964658678 -1.62505496 0.61068213 0.71541]] [[10 10 10 10]
 [10 10 10 10]] [[-24.383

In [ ]:
# vectorized implementation for the ESN basis idea
def _reservoir_out_batch(hidden_vec, res_wt, res_dim):
  # concatenate the hidden vector by rotating.
  # hidden_vec_batch = tf.repeat(hidden_vec, repeats=[res_dim], axis=-2)
  # hidden_vec_batch = tf.reshape(hidden_vec_batch, [-1, res_dim, *([1]*res_dim)])#, res_dim])# BxHxH
  # # hidden_vec_batch = tf.transpose()
  # print(list(range(0,res_dim)))
  # hidden_vec_batch = tf.roll(hidden_vec_batch, shift=range(0, res_dim), axis=range(1, res_dim+1))#range(2, res_dim+2))
  # print(hidden_vec_batch)
  # hidden_vec_batch = tf.reshape(hidden_vec_batch, [-1, res_dim, res_dim])
  # return hidden_vec_batch
  hidden_vec_batch = []
  for _ in range(res_dim):
    hidden_vec = tf.roll(hidden_vec, shift=[0, 1], axis=[0, 1])
    hidden_vec_batch.append(tf.expand_dims(hidden_vec, -1))
  hidden_vec_batch = tf.concat(hidden_vec_batch, axis=-1)
  return hidden_vec_batch * res_wt # [?, H, H]

def _projection_out_batch(input_vec, in_wt, in_dim, res_dim):
  if res_dim >= in_dim:  # case I:  res_dim >= in_dim
    # concat inputs to match the res_sim
    repeat = math.ceil(res_dim / in_dim) - 1 
    print('repeat: ', repeat)
    for i in range(repeat):
      if i == repeat - 1:
        input_vec = tf.concat([input_vec, input_vec[:, :(res_dim-repeat*(in_dim-1)-1)]], 
                              axis=-1)
      else:
        input_vec = tf.concat([input_vec, input_vec], axis=-1)
    input_vec_batch = []
    # rotate the input_vec
    for _ in range(res_dim):
      input_vec = tf.roll(input_vec, shift=[0, 1], axis=[-2, -1])
      input_vec_batch.append(tf.expand_dims(input_vec, -1))
    input_vec_batch = tf.concat(input_vec_batch, axis=-1)
    print('final in vec: ', input_vec_batch, in_wt)
    proj_vec = input_vec_batch * in_wt
  else: # Case II: res_dim < in_dim -->  FIX
    # 1. reapeat the input res_dim times with rot
    # 2. Do the hadamard product 3. Reshape & tf.reduce ; return HxH 
    repeat = math.ceil(in_dim/res_dim)-1
    print('repeat num :', repeat, res_dim, in_dim, input_vec.shape)
    # # Step 1. pad zeros
    # zeros_vec = tf.zeros([input_vec.shape[0], in_dim-repeat*(res_dim-1)-1])
    # # zeros_vec = tf.zeros([in_dim-repeat*(res_dim-1)-1])
    # print('zeros: ', zeros_vec, input_vec)
    # input_vec = tf.concat([input_vec, zeros_vec], axis=-1)
    print('check: ', input_vec) # BxI
    # Step 1. concat the Input with rotation
    input_vec_batch = []
    for _ in range(res_dim):
      input_vec = tf.roll(input_vec, shift=[0, 1], axis=[-2, -1])
      input_vec_batch.append(tf.expand_dims(input_vec, -1))
    input_vec_batch = tf.concat(input_vec_batch, axis=-1) # B x I x H
    # Step 2. The Hadamard product
    proj_vec_temp = input_vec_batch * in_wt # B x I x H
    print('intermediate shape: ', proj_vec_temp.shape)
    # Step 3. Reshape the proj_vec = B x H x H x (I/H)
    pv_final1 = proj_vec_temp[:, :res_dim, :res_dim] # B x H x H
    for r in range(1, in_dim//res_dim):
      pv_final1 = pv_final1 + proj_vec_temp[:, r*res_dim:(r+1)*res_dim, :res_dim]
    print('pv final1 shape: ', pv_final1.shape)
    if in_dim % res_dim != 0: # last part, pad zeros and add
      pv_final2 = proj_vec_temp[:, res_dim*(in_dim//res_dim):in_dim, :res_dim] # B x (I-H*r) x H
      paddings = tf.constant([[1,1,1], [1, 1, 2]])
      pv_final2 = tf.pad(pv_final2, paddings, mode='CONSTANT', name=None)
    proj_vec = pv_final2 #pv_final1 + pv_final2

    # proj_vec = tf.reshape(proj_vec, [-1, res_dim, res_dim, in_dim/res_dim])
    # Step 4. Reduce using tf.sum and squeeze
    # proj_vec = tf.math.reduce_sum(proj_vec, axis=-1)

    # # input_vec = tf.transpose(tf.expand_dims(input_vec, axis=-1)) #
    # input_vec = tf.reshape(input_vec, [input_vec.shape[0], repeat+1, res_dim])
    # input_vec = tf.transpose(input_vec, perm=[0, 2, 1])
    # # input_vec = tf.squeeze(input_vec, axis=-1)
    # print('check reshape: ', input_vec.shape) # B x H x r
    # #for _ in range(res_dim):
    # input_vec = tf.roll(input_vec, shift=[rot_num, 0], axis=[1, 2])
    # print('after roll: ', input_vec, tf.transpose(in_wt_vector))
    # proj_vec = tf.math.reduce_sum(input_vec*tf.transpose(in_wt_vector), axis=-1) # sum[(Hxr) * (Hxr)]
    # #print(proj_vec)
    # #proj_vec = input_vec
  return proj_vec

sess = tf.compat.v1.Session()
with sess.as_default():
  res_dim, in_dim = 4, 6# 6, 4
  if res_dim >= in_dim:
    in_wt_dim = res_dim
  else:
    print('res_dim < in_dim', in_dim/res_dim, math.ceil(in_dim/res_dim))
    # repeat_dim = math.ceil(in_dim/res_dim)
    # in_wt_dim = math.ceil(in_dim/res_dim) * res_dim 
    in_wt_dim = in_dim
  B = 1 # batch size
  hidden_vec = tf.random.normal([B, res_dim], seed=9)
  res_wt = tf.ones([res_dim, res_dim])*10
  
  roll_hidden_vec = tf.roll(hidden_vec, shift=[0, 2], axis=[0, 1])
  
  input_vec = tf.random.normal([B, in_dim], seed=9)
  # in_wt = tf.ones([res_dim*repeat_dim, res_dim])*10
  in_wt = tf.ones([in_wt_dim, res_dim])*10

  # tf.print(x1)
  # rot=1
  # z1 = _reservoir_out_batch(hidden_vec, res_wt, res_dim)
  z2 = _projection_out_batch(input_vec, in_wt, in_dim, res_dim)
  #cz1 = tf.concat([x1, y1], axis=-1)
  # print_op = tf.print("reservoir_out:", hidden_vec, res_wt, z1, z1.shape,
  #                       output_stream=sys.stdout)  
  print_op = tf.print("projection_out:", input_vec, in_wt, z2, z2.shape,
                         output_stream=sys.stdout)  
  with tf.control_dependencies([print_op]):  
    tripled_tensor =z1* 3  
  sess.run(tripled_tensor) 

res_dim < in_dim 1.5 2
repeat num : 1 4 6 (1, 6)
check:  Tensor("random_normal_515:0", shape=(1, 6), dtype=float32)
intermediate shape:  (1, 6, 4)
pv final1 shape:  (1, 4, 4)
projection_out: [[1.15507281 -0.491057307 0.444480509 -2.43835855 -1.99601936 -0.484241098]] [[10 10 10 10]
 [10 10 10 10]
 [10 10 10 10]
 [10 10 10 10]
 [10 10 10 10]
 [10 10 10 10]] [[[-24.383585 4.44480515 -4.91057301 11.5507278]
  [-19.9601936 -24.383585 4.44480515 -4.91057301]]] TensorShape([Dimension(1), Dimension(2), Dimension(4)])


In [ ]:
# vectorized implementation for the ESN basis idea: 3D mat
def _reservoir_out_batch3D(hidden_vec, res_wt, res_dim):
  # concatenate the hidden vector by rotating.
  # hidden_vec_batch = tf.repeat(hidden_vec, repeats=[res_dim], axis=-2)
  # hidden_vec_batch = tf.reshape(hidden_vec_batch, [-1, res_dim, *([1]*res_dim)])#, res_dim])# BxHxH
  # # hidden_vec_batch = tf.transpose()
  # print(list(range(0,res_dim)))
  # hidden_vec_batch = tf.roll(hidden_vec_batch, shift=range(0, res_dim), axis=range(1, res_dim+1))#range(2, res_dim+2))
  # print(hidden_vec_batch)
  # hidden_vec_batch = tf.reshape(hidden_vec_batch, [-1, res_dim, res_dim])
  # return hidden_vec_batch

  hidden_vec_batch = []
  for _ in range(res_dim):
    hidden_vec = tf.roll(hidden_vec, shift=[0, 1], axis=[0, 1])
    hidden_vec_batch.append(tf.expand_dims(hidden_vec, -1))
  hidden_vec_batch = tf.concat(hidden_vec_batch, axis=-1)
  return hidden_vec_batch * res_wt # [?, H, H]

  # def all_shifts(vector):
  #   length = tf.size(vector)
  #   repeated = tf.tile(vector, [vector.shape[0], length+1])  # [vector.shape[0],, length+1]
  #   print(repeated)
  #   return tf.reshape(repeated, [length, length+1])[:, :length]

  # return all_shifts(hidden_vec) * res_wt

def _projection_out_batch3D(input_vec, in_wt, in_dim, res_dim):
  if res_dim >= in_dim:  # case I:  res_dim >= in_dim
    # concat inputs to match the res_sim
    repeat = math.ceil(res_dim / in_dim) - 1 
    print('repeat: ', repeat)
    for i in range(repeat):
      if i == repeat - 1:
        input_vec = tf.concat([input_vec, input_vec[:, :(res_dim-repeat*(in_dim-1)-1)]], 
                              axis=-1)
      else:
        input_vec = tf.concat([input_vec, input_vec], axis=-1)
    input_vec_batch = []
    # rotate the input_vec
    for _ in range(res_dim):
      input_vec = tf.roll(input_vec, shift=[0, 1], axis=[-2, -1])
      input_vec_batch.append(tf.expand_dims(input_vec, -1))
    input_vec_batch = tf.concat(input_vec_batch, axis=-1)
    print('final in vec: ', input_vec_batch, in_wt)
    proj_vec = input_vec_batch * in_wt
  else: # Case II: res_dim < in_dim -->  FIX
    # 1. reapeat the input res_dim times with rot
    # 2. Do the hadamard product 3. Reshape & tf.reduce ; return HxH 
    repeat = math.ceil(in_dim/res_dim)-1
    print('repeat num :', repeat, res_dim, in_dim, input_vec.shape)
    # # Step 1. pad zeros
    # zeros_vec = tf.zeros([input_vec.shape[0], in_dim-repeat*(res_dim-1)-1])
    # # zeros_vec = tf.zeros([in_dim-repeat*(res_dim-1)-1])
    # print('zeros: ', zeros_vec, input_vec)
    # input_vec = tf.concat([input_vec, zeros_vec], axis=-1)
    print('check: ', input_vec) # BxI
    # Step 1. concat the Input with rotation
    input_vec_batch = []
    for _ in range(res_dim):
      input_vec = tf.roll(input_vec, shift=[0, 1], axis=[-2, -1])
      input_vec_batch.append(tf.expand_dims(input_vec, -1))
    input_vec_batch = tf.concat(input_vec_batch, axis=-1) # B x I x H
    # Step 2. The Hadamard product
    proj_vec_temp = input_vec_batch * in_wt # B x I x H
    print('intermediate shape: ', proj_vec_temp.shape)
    # Step 3. Reshape the proj_vec = B x H x H x (I/H)
    pv_final1 = proj_vec_temp[:, :res_dim, :res_dim] # B x H x H
    for r in range(1, in_dim//res_dim):
      pv_final1 = pv_final1 + proj_vec_temp[:, r*res_dim:(r+1)*res_dim, :res_dim]
    print('pv final1 shape: ', pv_final1.shape)
    if in_dim % res_dim != 0: # last part, pad zeros and add
      pv_final2 = proj_vec_temp[:, res_dim*(in_dim//res_dim):in_dim, :res_dim] # B x (I-H*r) x H
      paddings = tf.constant([[1,1,1], [1, 1, 2]])
      pv_final2 = tf.pad(pv_final2, paddings, mode='CONSTANT', name=None)
    proj_vec = pv_final2 #pv_final1 + pv_final2

    # proj_vec = tf.reshape(proj_vec, [-1, res_dim, res_dim, in_dim/res_dim])
    # Step 4. Reduce using tf.sum and squeeze
    # proj_vec = tf.math.reduce_sum(proj_vec, axis=-1)

    # # input_vec = tf.transpose(tf.expand_dims(input_vec, axis=-1)) #
    # input_vec = tf.reshape(input_vec, [input_vec.shape[0], repeat+1, res_dim])
    # input_vec = tf.transpose(input_vec, perm=[0, 2, 1])
    # # input_vec = tf.squeeze(input_vec, axis=-1)
    # print('check reshape: ', input_vec.shape) # B x H x r
    # #for _ in range(res_dim):
    # input_vec = tf.roll(input_vec, shift=[rot_num, 0], axis=[1, 2])
    # print('after roll: ', input_vec, tf.transpose(in_wt_vector))
    # proj_vec = tf.math.reduce_sum(input_vec*tf.transpose(in_wt_vector), axis=-1) # sum[(Hxr) * (Hxr)]
    # #print(proj_vec)
    # #proj_vec = input_vec
  return proj_vec

sess = tf.compat.v1.Session()
with sess.as_default():
  res_dim, in_dim = 4, 6# 6, 4
  if res_dim >= in_dim:
    in_wt_dim = res_dim
  else:
    print('res_dim < in_dim', in_dim/res_dim, math.ceil(in_dim/res_dim))
    # repeat_dim = math.ceil(in_dim/res_dim)
    # in_wt_dim = math.ceil(in_dim/res_dim) * res_dim 
    in_wt_dim = in_dim
  B = 1 # batch size
  hidden_vec = tf.random.normal([B, res_dim], seed=9)
  res_wt = tf.ones([res_dim, res_dim])*10
  
  roll_hidden_vec = tf.roll(hidden_vec, shift=[0, 2], axis=[0, 1])
  
  input_vec = tf.random.normal([B, in_dim], seed=9)
  # in_wt = tf.ones([res_dim*repeat_dim, res_dim])*10
  in_wt = tf.ones([in_wt_dim, res_dim])*10

  # tf.print(x1)
  # rot=1
  z1 = _reservoir_out_batch3D(hidden_vec, res_wt, res_dim)
  # z2 = _projection_out_batch3D(input_vec, in_wt, in_dim, res_dim)
  #cz1 = tf.concat([x1, y1], axis=-1)
  print_op = tf.print("reservoir_out:", hidden_vec, res_wt, z1, z1.shape,
                        output_stream=sys.stdout)  
  # print_op = tf.print("projection_out:", input_vec, in_wt, z2, z2.shape,
  #                        output_stream=sys.stdout)  
  with tf.control_dependencies([print_op]):  
    tripled_tensor =z1* 3  
  sess.run(tripled_tensor) 

res_dim < in_dim 1.5 2
reservoir_out: [[1.15507281 -0.491057307 0.444480509 -2.43835855]] [[10 10 10 10]
 [10 10 10 10]
 [10 10 10 10]
 [10 10 10 10]] [[[-24.383585 4.44480515 -4.91057301 11.5507278]
  [11.5507278 -24.383585 4.44480515 -4.91057301]
  [-4.91057301 11.5507278 -24.383585 4.44480515]
  [4.44480515 -4.91057301 11.5507278 -24.383585]]] TensorShape([Dimension(1), Dimension(4), Dimension(4)])


In [ ]:
# Basis3D 
def _reservoir_out_batch3D(hidden_vec, res_wt):
  # hidden_vec = BxH and res_wt = HxHxnum_basis
  # res_out = tf.matmul(hidden_vec, res_wt)
  print('shapes: ', hidden_vec.shape, res_wt.shape)
  # res_out = tf.matmul(hidden_vec, res_wt)

  res_wt = tf.transpose(res_wt, perm=[2,1,0]) # num_basis x H x H
  res_out = tf.matmul(res_wt, tf.transpose(hidden_vec))
  res_out = tf.transpose(res_out, perm=[2,1,0])
  return res_out # [B, H, num_basis]

def _projection_out_batch3D(input_vec, in_wt):
  # input_vec = BxI and in_wt = IxHxnum_basis
  # proj_out = tf.matmul(input_vec, in_wt)

  proj_out = tf.matmul(tf.transpose(in_wt, perm=[2,1,0]), tf.transpose(input_vec))
  proj_out = tf.transpose(proj_out, perm=[2,1,0])
  return proj_out # [B, H, num_basis]

sess = tf.compat.v1.Session()
with sess.as_default():
  res_dim, in_dim = 4, 6# 6, 4

  num_basis = 3

  B = 2 # batch size
  hidden_vec = tf.random.normal([B, res_dim], seed=9)
  res_wt = tf.ones([res_dim, res_dim, num_basis])*10
  # Create the mask for the reservoir basis matrix
  mask3D = []
  for rot in range(num_basis):
    _w_scr = tf.linalg.diag(tf.ones(res_dim))
    w_scr = tf.roll(_w_scr, shift=[rot, 0], axis=[0, 1])
    mask = w_scr # tf.cast(w_scr, dtype=dtype)
    print(mask.shape)
    mask3D.append(tf.expand_dims(mask, -1))
  mask = tf.concat(mask3D, axis=-1)
  
  res_wt = res_wt * mask
  
  input_vec = tf.random.normal([B, in_dim], seed=9)
  # in_wt = tf.ones([res_dim*repeat_dim, res_dim])*10
  in_wt = tf.ones([in_wt_dim, res_dim, num_basis])*10
  mask3D = []
  for rot in range(num_basis):
    dim0, dim1 = in_dim, res_dim
    if dim0 <= dim1:
      shape0, shape1 = dim0, dim1
      do_transpose = False
    else:
      shape0, shape1 = dim1, dim0
      do_transpose = True
    # assuming shape0 < shape1:
    repeat = math.ceil(shape1 / shape0) - 1
    w_scr = tf.eye(shape0)
    for i in range(repeat):
      if i == repeat - 1:  # truncate
        w_diag = tf.eye(shape0)
        w_scr = tf.concat([w_scr, w_diag[:, :(shape1 - shape0)]], axis=1)
      else:
        w_scr = tf.concat([w_scr, tf.eye(shape0)], axis=1)
    w_scr = tf.roll(w_scr, shift=[rot, 0], axis=[0, 1])
    # the transpose condition
    if do_transpose:
      w_scr = tf.transpose(w_scr, perm=[1, 0])
    mask = w_scr #tf.cast(w_scr, dtype=dtype)
    mask3D.append(tf.expand_dims(mask, -1))
  mask = tf.concat(mask3D, axis=-1)
  print('in mask: ', mask.shape)
  in_wt = in_wt * mask
  # v = in_wt
  # v_norm2 = tf.sqrt(tf.reduce_sum(tf.square(v)))
  # is_norm_0 = tf.equal(v_norm2, 0)
  # print(is_norm_0)
  # v = v * 1.0/(v_norm2 +is_norm_0)
  # print(v.shape)

  z1 = _reservoir_out_batch3D(hidden_vec, res_wt)
  print_op = tf.print("reservoir_out:", hidden_vec, res_wt, z1, z1.shape,
                        output_stream=sys.stdout)
  
  # z2 = _projection_out_batch3D(input_vec, in_wt)
  # print_op = tf.print("projection_out:", input_vec, in_wt, z2, z2.shape,
  #                        output_stream=sys.stdout)  
  with tf.control_dependencies([print_op]):  
    tripled_tensor =z1* 3  
  sess.run(tripled_tensor) 

(4, 4)
(4, 4)
(4, 4)
in mask:  (6, 4, 3)
shapes:  (2, 4) (4, 4, 3)
reservoir_out: [[1.15507281 -0.491057307 0.444480509 -2.43835855]
 [-1.99601936 -0.484241098 -0.491944462 0.527831]] [[[10 0 0]
  [0 0 0]
  [0 0 10]
  [0 10 0]]

 [[0 10 0]
  [10 0 0]
  [0 0 0]
  [0 0 10]]

 [[0 0 10]
  [0 10 0]
  [10 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 10]
  [0 10 0]
  [10 0 0]]] [[[11.5507278 -4.91057301 4.44480515]
  [-4.91057301 4.44480515 -24.383585]
  [4.44480515 -24.383585 11.5507278]
  [-24.383585 11.5507278 -4.91057301]]

 [[-19.9601936 -4.84241104 -4.91944456]
  [-4.84241104 -4.91944456 5.2783103]
  [-4.91944456 5.2783103 -19.9601936]
  [5.2783103 -19.9601936 -4.84241104]]] TensorShape([Dimension(2), Dimension(4), Dimension(3)])


In [ ]:
# Basis3D 
def _reservoir_out_batch3D(hidden_vec, res_wt):
  # hidden_vec = BxH and res_wt = HxHxnum_basis
  # res_out = tf.matmul(hidden_vec, res_wt)
  print('shapes: ', hidden_vec.shape, res_wt.shape)
  # res_out = tf.matmul(hidden_vec, res_wt)

  res_wt = tf.transpose(res_wt, perm=[2,1,0]) # num_basis x H x H
  res_out = tf.matmul(res_wt, tf.transpose(hidden_vec)) # num_basis x H x B
  res_out = tf.transpose(res_out, perm=[2,1,0])
  return res_out # [B, H, num_basis]

def _projection_out_batch3D(input_vec, in_wt):
  # input_vec = BxI and in_wt = IxHxnum_basis
  # proj_out = tf.matmul(input_vec, in_wt)

  proj_out = tf.matmul(tf.transpose(in_wt, perm=[2,1,0]), tf.transpose(input_vec))
  proj_out = tf.transpose(proj_out, perm=[2,1,0])
  return proj_out # [B, H, num_basis]

sess = tf.compat.v1.Session()
with sess.as_default():
  res_dim, in_dim = 4, 6# 6, 4

  num_basis = 3

  B = 2 # batch size
  hidden_vec = tf.random.normal([B, res_dim], seed=9)
  res_wt = tf.ones([res_dim, res_dim, num_basis])*10
  # Create the mask for the reservoir basis matrix
  mask3D = []
  for rot in range(num_basis):
    _w_scr = tf.linalg.diag(tf.ones(res_dim))
    w_scr = tf.roll(_w_scr, shift=[rot, 0], axis=[0, 1])
    mask = w_scr # tf.cast(w_scr, dtype=dtype)
    print(mask.shape)
    mask3D.append(tf.expand_dims(mask, -1))
  mask = tf.concat(mask3D, axis=-1)
  
  res_wt = res_wt * mask

  z1 = _reservoir_out_batch3D(hidden_vec, res_wt)
  print_op = tf.print("reservoir_out:", hidden_vec, tf.transpose(mask, perm=[2,1,0]), res_wt, z1, z1.shape,
                        output_stream=sys.stdout)
   
  with tf.control_dependencies([print_op]):  
    tripled_tensor =z1* 3  
  sess.run(tripled_tensor) 

(6, 6)
(6, 6)
(6, 6)
shapes:  (2, 6) (6, 6, 3)
reservoir_out: [[1.15507281 -0.491057307 0.444480509 -2.43835855 -1.99601936 -0.484241098]
 [-0.491944462 0.527831 1.04352212 3.02801514 -0.460363448 0.412836105]] [[[1 0 0 0 0 0]
  [0 1 0 0 0 0]
  [0 0 1 0 0 0]
  [0 0 0 1 0 0]
  [0 0 0 0 1 0]
  [0 0 0 0 0 1]]

 [[0 1 0 0 0 0]
  [0 0 1 0 0 0]
  [0 0 0 1 0 0]
  [0 0 0 0 1 0]
  [0 0 0 0 0 1]
  [1 0 0 0 0 0]]

 [[0 0 1 0 0 0]
  [0 0 0 1 0 0]
  [0 0 0 0 1 0]
  [0 0 0 0 0 1]
  [1 0 0 0 0 0]
  [0 1 0 0 0 0]]] [[[10 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 10]
  [0 10 0]]

 [[0 10 0]
  [10 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 10]]

 [[0 0 10]
  [0 10 0]
  [10 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 10]
  [0 10 0]
  [10 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 10]
  [0 10 0]
  [10 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 10]
  [0 10 0]
  [10 0 0]]] [[[11.5507278 -4.91057301 4.44480515]
  [-4.91057301 4.44480515 -24.383585]
  [4.44480515 -24.383585 -19.

In [ ]:
res_wt = tf.ones([4, 4, 4])
scalar_vec = tf.constant([2,10,200.0,1000])
scalar_vec = tf.reshape(scalar_vec, [1, 4])
res_scalar = tf.multiply(scalar_vec, res_wt)
print(res_scalar, res_wt)
sess = tf.compat.v1.Session()
with sess.as_default():
  result = sess.run([res_scalar, res_wt, scalar_vec])
  #print([:, :, 0])#, res_wt[:, :, 2])

Tensor("Mul_1318:0", shape=(4, 4, 4), dtype=float32) Tensor("ones_814:0", shape=(4, 4, 4), dtype=float32)


In [ ]:
print(result[0][:, :,0], result[0][:, :, 2], result[1][:,:,0], result[1][:,:,2])

[[2. 2. 2. 2.]
 [2. 2. 2. 2.]
 [2. 2. 2. 2.]
 [2. 2. 2. 2.]] [[200. 200. 200. 200.]
 [200. 200. 200. 200.]
 [200. 200. 200. 200.]
 [200. 200. 200. 200.]] [[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]] [[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]


In [ ]:
  def all_shifts(vector):
    length = tf.size(vector)
    repeated = tf.tile(vector, [length+1])
    return tf.reshape(repeated, [length, length+1])[:, :length]

In [ ]:
# Basis efficient implementation
def all_shifts(hidden_state, H):
  # H = hidden_vec.shape[-1]
  repeated = tf.tile(hidden_state, [1,H+1])
  out = tf.reshape(repeated, [-1, H, H+1])[:,:,:H]
  return out

def _reservoir_out(hidden_vec, res_wt, res_dim):
  print('shapes: ', hidden_vec.shape, res_wt.shape)
  return all_shifts(hidden_vec, res_dim) * res_wt

def _projection_out(input_vec, in_wt, in_dim, res_dim):
  if res_dim >= in_dim:  # case I:  res_dim >= in_dim
    # concat inputs to match the res_sim
    repeat = math.ceil(res_dim / in_dim) - 1 
    print('repeat: ', repeat, res_dim, in_dim, input_vec.shape)
    for i in range(repeat):
      if i == repeat - 1:
        # input_vec = tf.concat([input_vec, input_vec[:, :(res_dim-repeat*(in_dim-1)-1)]], 
        #                       axis=-1)
        input_vec = tf.concat([input_vec, input_vec[:, :(res_dim-(repeat*in_dim))]], 
                              axis=-1)
      else:
        input_vec = tf.concat([input_vec, input_vec], axis=-1)
    print('input vec: ', input_vec.shape)
    input_vec_batch = all_shifts(input_vec, res_dim)
    print('final in vec: ', input_vec_batch, in_wt)
    proj_vec = input_vec_batch * in_wt
  else: # Case II: res_dim < in_dim -->  FIX
    print('Case res_dim < in_dim not implemented', res_dim, in_dim)
  return proj_vec

sess = tf.compat.v1.Session()
with sess.as_default():
  res_dim, in_dim = 4, 4# 6, 4

  num_basis = res_dim #3

  B = 2 # batch size
  hidden_vec = tf.random.normal([B, res_dim], seed=9)
  res_wt = tf.ones([num_basis, res_dim])*10

  # z1 = _reservoir_out(hidden_vec, res_wt, res_dim)
  # print_op = tf.print("reservoir_out:", hidden_vec, res_wt, z1, z1.shape,
  #                       output_stream=sys.stdout)
  input_vec = tf.random.normal([B, in_dim], seed=10)
  in_wt = tf.ones([num_basis, res_dim])*10
  z2 = _projection_out(input_vec, in_wt, in_dim, res_dim)
  print_op = tf.print("projection_out:", input_vec, in_wt, z2, z2.shape,
                         output_stream=sys.stdout) 
  
  with tf.control_dependencies([print_op]):  
    tripled_tensor =z1* 3  
  sess.run(tripled_tensor) 

NameError: ignored

In [ ]:
def all_shifts(vector):
  length = tf.size(vector)
  repeated = tf.tile(vector, [length+1])
  # out = tf.reshape(repeated, [length, length+1])[:, :length]
  out = tf.reshape(repeated, [length, length+1])
  return repeated, out

h = tf.constant([1,2,3,4])
repeated, out = all_shifts(h)

with tf.compat.v1.Session() as sess:
  res = sess.run([repeated, out])
  print(res)


[array([1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4],
      dtype=int32), array([[1, 2, 3, 4, 1],
       [2, 3, 4, 1, 2],
       [3, 4, 1, 2, 3],
       [4, 1, 2, 3, 4]], dtype=int32)]


In [ ]:
H = 4 # hidden size
B = 2 # batch size

def all_shifts(hidden_state):
  repeated = tf.tile(hidden_state, [1,H+1])
  # out = tf.reshape(repeated, [length, length+1])[:, :length]
  # out = tf.reshape(repeated, [length, length+1])
  out = tf.reshape(repeated, [-1, H, H+1])[:,:,:H]
  return repeated, out


h = tf.reshape(tf.range(8),[B,H])
# h = tf.constant([[0,1,2,3],[4,5,6,7]])
repeated, out = all_shifts(h)

with tf.compat.v1.Session() as sess:
  res = sess.run([repeated, out])
  print(res[0], res[0].shape)
  print(res[1], res[1].shape)


[[0 1 2 3 0 1 2 3 0 1 2 3 0 1 2 3 0 1 2 3]
 [4 5 6 7 4 5 6 7 4 5 6 7 4 5 6 7 4 5 6 7]] (2, 20)
[[[0 1 2 3]
  [1 2 3 0]
  [2 3 0 1]
  [3 0 1 2]]

 [[4 5 6 7]
  [5 6 7 4]
  [6 7 4 5]
  [7 4 5 6]]] (2, 4, 4)


In [ ]:
tf.linalg.diag([1.63685679, 1.00188148, 1.25753987, 0.450119644])

<tf.Tensor 'diag_183:0' shape=(4, 4) dtype=float32>

In [ ]:
tf.diag_part(tf.random.normal((5, 5)))

<tf.Tensor 'DiagPart:0' shape=(5,) dtype=float32>

In [ ]:
import numpy as np 

In [ ]:
#W = np.array([[0, 1, 0, 0, 1], [1, 0, 1,0, 0], [0,1,0,1,0],[0,0,1,0,1], [1,0,1,0,0]])

In [ ]:
D = np.diag([2,2,2,2])

In [ ]:
# W = np.array([[0, 1, 0, 1], [1, 0, 1, 0], [0,1,0,1], [1,0,1,0]]) # 4 x 4
W = np.array([[0, 1, 0, 1], [1, 0, 1, 0], [0,1,0,1], [1,0,1,0]])
# W = np.array([[0, 0, 0, 1], [1, 0, 0, 0], [0,1,0,0], [1,0,0,0]])
D = np.diag([2,2,2,2])
inv_D = np.sqrt(np.diag(np.ones(4)*1/2.0))
# inv_D = np.sqrt(np.diag(np.ones(4)))

#print(inv_D)
#L = np.eye(4) - np.matmul(np.matmul(inv_D, W), inv_D)
L = D-W
eigval, eigvec = np.linalg.eig(L)
print(D-W)
print(L,' \n', eigval)
print(eigvec)
#print(np.linalg.inv(eigvec))

[[ 2 -1  0 -1]
 [-1  2 -1  0]
 [ 0 -1  2 -1]
 [-1  0 -1  2]]
[[ 2 -1  0 -1]
 [-1  2 -1  0]
 [ 0 -1  2 -1]
 [-1  0 -1  2]]  
 [-2.22044605e-16  2.00000000e+00  4.00000000e+00  2.00000000e+00]
[[ 5.00000000e-01  7.07106781e-01 -5.00000000e-01  2.77555756e-17]
 [ 5.00000000e-01 -2.02962647e-16  5.00000000e-01 -7.07106781e-01]
 [ 5.00000000e-01 -7.07106781e-01 -5.00000000e-01  2.86262916e-16]
 [ 5.00000000e-01  1.02348685e-16  5.00000000e-01  7.07106781e-01]]


In [ ]:
import scipy
np.set_printoptions(precision=4, suppress=True)

In [ ]:
#np.printoptions(precision=3, suppress=True)
import numpy as np
def DFT_matrix(N):
    i, j = np.meshgrid(np.arange(N), np.arange(N))
    omega = np.exp( - 2 * np.pi * 1J / N )
    W = np.power( omega, i * j ) / np.sqrt(N)
    return W

mat_dft = DFT_matrix(4)
print(mat_dft)
# print(np.linalg.inv(mat_dft))
# print(np.linalg.eig(mat_dft))
# print(np.linalg.inv(mat_dft))

[[ 0.5+0.j   0.5+0.j   0.5+0.j   0.5+0.j ]
 [ 0.5+0.j   0. -0.5j -0.5-0.j  -0. +0.5j]
 [ 0.5+0.j  -0.5-0.j   0.5+0.j  -0.5-0.j ]
 [ 0.5+0.j  -0. +0.5j -0.5-0.j   0. -0.5j]]


In [ ]:
eigval, eigvec = np.linalg.eig(L)
print(np.matmul(mat_dft, L))

[[ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 1.+0.j  0.-1.j -1.-0.j -0.+1.j]
 [ 2.+0.j -2.-0.j  2.+0.j -2.-0.j]
 [ 1.+0.j -0.+1.j -1.-0.j  0.-1.j]]


In [ ]:
lambda_mat = np.diag(eigval)
print(np.matmul(mat_dft, lambda_mat).transpose())

[[-0.+0.j -0.+0.j -0.+0.j -0.+0.j]
 [ 1.+0.j  0.-1.j -1.-0.j -0.+1.j]
 [ 2.+0.j -2.-0.j  2.+0.j -2.-0.j]
 [ 1.+0.j -0.+1.j -1.-0.j  0.-1.j]]


In [ ]:
W = np.array([[0, 1, 0, 1], [1, 0, 1, 0], [0,1,0,1], [1,0,1,0]])
D = np.diag([2,2,2,2])
inv_D = np.sqrt(np.diag(np.ones(4)*1/2.0))
#print(inv_D)
L = np.eye(4) - np.matmul(np.matmul(inv_D, W), inv_D)
#L = D-W
eigval, eigvec = np.linalg.eig(L)
print(eigvec)
print(mat_dft)
mat_dft = DFT_matrix(4)
print(np.matmul(L, mat_dft))
lambda_mat = np.diag(eigval)
print(np.matmul(mat_dft, lambda_mat))
print(np.linalg.qr(eigvec))
# print('****************8')
# print(np.matmul(L, eigvec))
# print(np.matmul(eigvec, lambda_mat))

# print('*****************')
# print(np.matmul(eigvec.transpose(), L))
# print(np.matmul(eigvec.transpose(), lambda_mat))

# print('*******NO TRANPOSE**********')
# print(np.matmul(eigvec, L))
# print(np.matmul(eigvec, lambda_mat))

[[-0.5     0.7071 -0.5     0.    ]
 [ 0.5     0.     -0.5    -0.7071]
 [-0.5    -0.7071 -0.5    -0.    ]
 [ 0.5    -0.     -0.5     0.7071]]
[[ 0.5+0.j   0.5+0.j   0.5+0.j   0.5+0.j ]
 [ 0.5+0.j   0. -0.5j -0.5-0.j  -0. +0.5j]
 [ 0.5+0.j  -0.5-0.j   0.5+0.j  -0.5-0.j ]
 [ 0.5+0.j  -0. +0.5j -0.5-0.j   0. -0.5j]]
[[-0. +0.j   0.5+0.j   1. +0.j   0.5+0.j ]
 [-0. +0.j   0. -0.5j -1. -0.j  -0. +0.5j]
 [-0. +0.j  -0.5-0.j   1. +0.j  -0.5-0.j ]
 [-0. +0.j  -0. +0.5j -1. -0.j   0. -0.5j]]
[[ 1. +0.j   0.5+0.j  -0. +0.j   0.5+0.j ]
 [ 1. +0.j   0. -0.5j  0. +0.j  -0. +0.5j]
 [ 1. +0.j  -0.5-0.j  -0. -0.j  -0.5-0.j ]
 [ 1. +0.j  -0. +0.5j  0. +0.j   0. -0.5j]]
(array([[-0.5   , -0.7071, -0.5   , -0.    ],
       [ 0.5   , -0.    , -0.5   , -0.7071],
       [-0.5   ,  0.7071, -0.5   , -0.    ],
       [ 0.5   , -0.    , -0.5   ,  0.7071]]), array([[ 1., -0.,  0., -0.],
       [ 0., -1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.]]))


In [ ]:
++W = np.array([[0, 1, 0, 0, 1], [1, 0, 1,0, 0], [0,1,0,1,0],[0,0,1,0,1], [1,0,0,1,0]])
D = np.diag([2,2,2,2,2])
L = D - W
inv_D = np.sqrt(np.diag(np.ones(5)*1/2.0))
print(inv_D)
L = np.eye(5) - np.matmul(np.matmul(inv_D, W), inv_D)
eigval, eigvec = np.linalg.eig(L)
print(W, L, eigval, eigvec)


[[0.70710678 0.         0.         0.         0.        ]
 [0.         0.70710678 0.         0.         0.        ]
 [0.         0.         0.70710678 0.         0.        ]
 [0.         0.         0.         0.70710678 0.        ]
 [0.         0.         0.         0.         0.70710678]]
[[0 1 0 0 1]
 [1 0 1 0 0]
 [0 1 0 1 0]
 [0 0 1 0 1]
 [1 0 0 1 0]] [[ 1.  -0.5  0.   0.  -0.5]
 [-0.5  1.  -0.5  0.   0. ]
 [ 0.  -0.5  1.  -0.5  0. ]
 [ 0.   0.  -0.5  1.  -0.5]
 [-0.5  0.   0.  -0.5  1. ]] [ 1.80901699e+00  6.90983006e-01 -1.61562165e-16  1.80901699e+00
  6.90983006e-01] [[-0.63245553  0.63245553 -0.4472136  -0.22487771  0.14475218]
 [ 0.51166727  0.19543951 -0.4472136  -0.16552528 -0.54080397]
 [-0.19543951 -0.51166727 -0.4472136   0.49270325 -0.47898741]
 [-0.19543951 -0.51166727 -0.4472136  -0.63168532  0.24477347]
 [ 0.51166727  0.19543951 -0.4472136   0.52938507  0.63026573]]


In [ ]:
W = np.array([[0, 1, 0, 0, 1.2], [1, 0, 1,0, 0], [0,1,0,1,0],[0,0,1,0,1], [1.2,0,0,1,0]])
D = np.diag([2.2,2,2,2,2.2])
L = D - W
eigval, eigvec = np.linalg.eig(L)
print(W, D, L, eigval,'\n', eigvec)

[[0.  1.  0.  0.  1.2]
 [1.  0.  1.  0.  0. ]
 [0.  1.  0.  1.  0. ]
 [0.  0.  1.  0.  1. ]
 [1.2 0.  0.  1.  0. ]] [[2.2 0.  0.  0.  0. ]
 [0.  2.  0.  0.  0. ]
 [0.  0.  2.  0.  0. ]
 [0.  0.  0.  2.  0. ]
 [0.  0.  0.  0.  2.2]] [[ 2.2 -1.   0.   0.  -1.2]
 [-1.   2.  -1.   0.   0. ]
 [ 0.  -1.   2.  -1.   0. ]
 [ 0.   0.  -1.   2.  -1. ]
 [-1.2  0.   0.  -1.   2.2]] [-1.33226763e-15  3.92065556e+00  3.61803399e+00  1.38196601e+00
  1.47934444e+00] 
 [[-4.47213595e-01 -6.27188637e-01  1.95439508e-01  5.11667274e-01
  -3.26549252e-01]
 [-4.47213595e-01  3.26549252e-01 -5.11667274e-01 -1.95439508e-01
  -6.27188637e-01]
 [-4.47213595e-01 -1.21020360e-15  6.32455532e-01 -6.32455532e-01
  -4.28024111e-15]
 [-4.47213595e-01 -3.26549252e-01 -5.11667274e-01 -1.95439508e-01
   6.27188637e-01]
 [-4.47213595e-01  6.27188637e-01  1.95439508e-01  5.11667274e-01
   3.26549252e-01]]


In [ ]:
W = np.array([[0, 0, 0, 0, 1], [1, 0, 0,0, 0], [0,1,0,0,0],[0,0,1,0,0], [0,0,0,1,0]])
D = np.diag([2,2,2,2,2])/2
L = D - W
eigval, eigvec = np.linalg.eig(L)
print(W, D, L)
print(eigval, eigvec)

[[0 0 0 0 1]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]] [[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]] [[ 1.  0.  0.  0. -1.]
 [-1.  1.  0.  0.  0.]
 [ 0. -1.  1.  0.  0.]
 [ 0.  0. -1.  1.  0.]
 [ 0.  0.  0. -1.  1.]]
[-1.20186557e-16+0.j          6.90983006e-01+0.95105652j
  6.90983006e-01-0.95105652j  1.80901699e+00+0.58778525j
  1.80901699e+00-0.58778525j] [[ 0.4472136+0.j         -0.4472136+0.j         -0.4472136-0.j
  -0.3618034+0.26286556j -0.3618034-0.26286556j]
 [ 0.4472136+0.j         -0.1381966-0.4253254j  -0.1381966+0.4253254j
   0.1381966-0.4253254j   0.1381966+0.4253254j ]
 [ 0.4472136+0.j          0.3618034-0.26286556j  0.3618034+0.26286556j
   0.1381966+0.4253254j   0.1381966-0.4253254j ]
 [ 0.4472136+0.j          0.3618034+0.26286556j  0.3618034-0.26286556j
  -0.3618034-0.26286556j -0.3618034+0.26286556j]
 [ 0.4472136+0.j         -0.1381966+0.4253254j  -0.1381966-0.4253254j
   0.4472136+0.j          0.4472136-0.j   

In [ ]:
H = 5
shape = [H, H]
_w_adj = tf.linalg.diag(tf.ones(shape[-1]))
w_adj = tf.roll(
    _w_adj, shift=[-1, 0], axis=[0, 1]) + tf.roll(
        _w_adj, shift=[1, 0], axis=[0, 1])
mask = w_adj + _w_adj

w_diag = tf.reduce_sum(w_adj, axis=-1)

inv_D = tf.linalg.diag(tf.sqrt(1/w_diag))
print(inv_D)
lap = tf.eye(shape[-1]) - tf.linalg.matmul(tf.linalg.matmul(inv_D, w_adj), inv_D)

# laplacian complementary
eigval, eigvec = tf.linalg.eigh(lap)
eigvec_inv = tf.linalg.inv(eigvec)
ortho_eigvec = tf.matmul(tf.transpose(eigvec), eigvec)
# zip(eigval, eigvec)
#eigval = tf.expand_dims(eigval, -1)
print(eigval)
eigval_c =  tf.reverse(eigval, axis=[0])

lap_c = tf.matmul(tf.matmul(eigvec, tf.linalg.diag(eigval_c)), tf.transpose(eigvec))
eigval_cc, eigvec_cc = tf.linalg.eigh(lap_c)

mask_c = tf.ones(lap_c.shape)  # tf.cast(lap_c, dtype=tf.float32)
sess = tf.compat.v1.Session()
with sess.as_default():
  result = sess.run([w_adj, w_diag, inv_D, mask, lap, eigval, eigvec, eigvec_inv, ortho_eigvec, eigval_c, lap_c, eigval_cc, eigvec_cc, mask_c])


Tensor("diag_7:0", shape=(5, 5), dtype=float32)
Tensor("SelfAdjointEigV2:0", shape=(5,), dtype=float32)


In [ ]:
names = ['w_adj', 'w_diag', 'inv_D', 'mask', 'lap', 'eigval', 'eigvec', 
         'eigvec_inv', 'ortho_eigvec', 'eigval_c', 'lap_c', 'eigval_cc',
         'eigvec_cc', 'mask_c']
res_dict = {n:m for n,m in zip(names, result)}
# for i, v in enumerate(result):
#   if i>5:
#     print(i, v)
#     print('\n')
np.set_printoptions(precision=4, suppress=True)
for n in ['lap', 'lap_c', 'eigval', 'eigval_c']:
  print(n)
  #print('Lap-chain')
  print(res_dict[n])

lap
[[ 1.  -0.5  0.   0.  -0.5]
 [-0.5  1.  -0.5  0.   0. ]
 [ 0.  -0.5  1.  -0.5  0. ]
 [ 0.   0.  -0.5  1.  -0.5]
 [-0.5  0.   0.  -0.5  1. ]]
lap_c
[[ 0.9146  0.2236  0.2236  0.2236  0.2236]
 [ 0.2236  1.2236  0.6281 -0.1809 -0.0854]
 [ 0.2236  0.6281  0.8191  0.3191 -0.1809]
 [ 0.2236 -0.1809  0.3191  0.8191  0.6281]
 [ 0.2236 -0.0854 -0.1809  0.6281  1.2236]]
eigval
[0.    0.691 0.691 1.809 1.809]
eigval_c
[1.809 1.809 0.691 0.691 0.   ]


In [ ]:
# for input weight: scrbd topology
sessibd = tf.compat.v1.Session()  

def get_complementary_topology_SVD(lap, dtype):
  # take the svd
  sigma, U, V = tf.linalg.svd(lap)
  print(U, sigma, V)
  # reverse the sigma
  # sigma = tf.expand_dims(sigma,-1)
  sigma_c = tf.reverse(sigma, axis=[0])#
  print('resverse: ', sigma_c)
  # # get the complementary laplacian
  # lap_c = tf.matmul(U, )
  lap_c = tf.matmul(U, tf.matmul(tf.linalg.diag(sigma_c), V, adjoint_b=True))
  mask_c = tf.ones(lap_c.shape)
  # mask_c = tf.cast(mask_c, dtype=dtype)
  # lap_c = tf.cast(lap_c, dtype=dtype)
  return lap_c, mask_c

with sessibd.as_default(): # initialize the reservoir weights
      shape = (6,6)
      # shape0, shape1 = shape[-2], shape[-1]
      dim0, dim1 = shape[-2], shape[-1]
      if dim0 <= dim1:
        shape0, shape1 = dim0, dim1
        do_transpose = False
      else:
        shape0, shape1 = dim1, dim0
        do_transpose = True
      # assuming shape0 < shape1:
      repeat = math.ceil(shape1/shape0) - 1
      w_scr = tf.eye(shape0)
      for i in range(repeat):
        if i == repeat - 1: # truncate
          w_diag = tf.eye(shape0)
          w_scr = tf.concat([w_scr, w_diag[:, :(shape1-shape0)]], axis=1)
        else:
          w_scr = tf.concat([w_scr, tf.eye(shape0) ], axis=1)
      D_mat = w_scr
      adj_mat = tf.roll(w_scr, shift=[-1, 0], axis=[0, 1]) + tf.roll(w_scr, shift=[1, 0], axis=[0, 1])
      w_scr = adj_mat + D_mat
      # the transpose condition
      if do_transpose:
        w_scr = tf.transpose(w_scr, perm=[1, 0])

      mask = w_scr # tf.cast(w_scr, dtype=dtype)
      # should have v and mask
      D_mat = D_mat*tf.reduce_sum(adj_mat, axis=-1)[0]  # scalar
      v = D_mat - adj_mat

      # SVD code check
      lap_c, mask_c = get_complementary_topology_SVD(v, 'dtype')

      print_op = tf.print("tensors:", mask,v, lap_c, mask_c,
                          output_stream=sys.stdout)  
      with tf.control_dependencies([print_op]):  
        tripled_tensor = w_scr * 3  
      sessibd.run(tripled_tensor) 

Tensor("Svd_1:1", shape=(6, 6), dtype=float32) Tensor("Svd_1:0", shape=(6,), dtype=float32) Tensor("Svd_1:2", shape=(6, 6), dtype=float32)
resverse:  Tensor("ReverseV2:0", shape=(6,), dtype=float32)
tensors: [[1 1 0 0 0 1]
 [1 1 1 0 0 0]
 [0 1 1 1 0 0]
 [0 0 1 1 1 0]
 [0 0 0 1 1 1]
 [1 0 0 0 1 1]] [[2 -1 0 0 0 -1]
 [-1 2 -1 0 0 0]
 [0 -1 2 -1 0 0]
 [0 0 -1 2 -1 0]
 [0 0 0 -1 2 -1]
 [-1 0 0 0 -1 2]] [[2.00000119 1.00000036 2.61051468e-07 -5.62475918e-07 -2.91244419e-07 1.00000024]
 [1.00000048 2.00000072 1.00000036 -2.42577386e-07 -2.09764451e-07 -4.79094027e-08]
 [3.69323089e-07 1.00000036 2.00000072 1.00000048 -4.37245376e-07 1.41842165e-07]
 [-3.83662e-07 -1.72035115e-07 1.00000036 2.00000143 1.00000012 4.45081518e-08]
 [-2.53515111e-07 -1.01492823e-07 -5.07787661e-07 1.00000036 2.00000072 1.00000083]
 [1.00000024 -1.45243405e-07 -1.34305751e-07 -1.23368096e-07 1.00000072 2.00000095]] [[1 1 1 1 1 1]
 [1 1 1 1 1 1]
 [1 1 1 1 1 1]
 [1 1 1 1 1 1]
 [1 1 1 1 1 1]
 [1 1 1 1 1 1]]


In [ ]:
wt = 'spec_lap_chain'
if 'spec' in wt:
  print(wt[-1])

n


In [ ]:
R = 2048
def get_closest_int(R):
  # R = f1 x f2, where f1 f2 are the closest
  f1 = np.ceil(np.sqrt(R))
  while(True):
    if f1==R:
      f2 = 1
      break
    elif R%f1==0:
      f2 = R/f1
      break
    else:
      f1 += 1
  return [int(f1), int(f2)]
grid_dim = get_closest_int(R)
print(grid_dim)
G = nx.grid_graph(grid_dim, periodic=True)
A = nx.to_numpy_matrix(G)
print(A, np.sum(A, axis=-1))

[64, 32]
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]] [[4.]
 [4.]
 [4.]
 ...
 [4.]
 [4.]
 [4.]]


In [ ]:
# Laplacian Grid topology for reservoir
sess2 = tf.compat.v1.Session()  

def get_lap_grid(R):
    grid_dim = get_closest_int(R)
    print(grid_dim)
    G = nx.grid_graph(grid_dim, periodic=True)
    A = nx.to_numpy_matrix(G)
    A = tf.convert_to_tensor(A)
    D = tf.reduce_sum(A, axis=-1)
    L = tf.linalg.diag(D)-A
    if R %2==0: # even
      mask = tf.linalg.diag(tf.ones(A.shape[-1], dtype=A.dtype)) + A
    else: # odd
      mask = A
    return L, mask

with sess2.as_default(): # initialize the reservoir weights
    shape = (6, 6)
    R = shape[-1]
    L, mask = get_lap_grid(R)
    print('L:', L.shape)
    # L = tf.convert_to_tensor(L) # dtype=dtype
    # take the svd
    eigval, eigvec = tf.linalg.eigh(L)
    print(eigval, eigvec)
    # print_op = tf.print("tensors:", mask, L, eigval, eigvec, 
    #                     output_stream=sys.stdout) 
    print_op = tf.print("Lap-grid:\n", L,
                        output_stream=sys.stdout)  
    with tf.control_dependencies([print_op]):  
      tripled_tensor = w_scr * 3
    sess2.run(tripled_tensor) 

[3, 2]
L: (6, 6)
Tensor("SelfAdjointEigV2:0", shape=(6,), dtype=float64) Tensor("SelfAdjointEigV2:1", shape=(6, 6), dtype=float64)
Lap-grid:
 [[3 -1 -1 -1 0 0]
 [-1 3 -1 0 -1 0]
 [-1 -1 3 0 0 -1]
 [-1 0 0 3 -1 -1]
 [0 -1 0 -1 3 -1]
 [0 0 -1 -1 -1 3]]


In [ ]:
# Laplacian Grid topology for reservoir
sess2 = tf.compat.v1.Session()  

with sess2.as_default(): # initialize the reservoir weights
    shape = (6,6)
    # shape0, shape1 = shape[-2], shape[-1]
    dim0, dim1 = shape[-2], shape[-1]
    if dim0 <= dim1:
      shape0, shape1 = dim0, dim1
      do_transpose = False
    else:
      shape0, shape1 = dim1, dim0
      do_transpose = True
    # assuming shape0 < shape1:
    repeat = math.ceil(shape1/shape0) - 1
    L, mask = get_lap_grid(R=shape0)
    for i in range(repeat):
      tempL, tempM = get_lap_grid(R=shape0)
      if i == repeat - 1: # truncate
        L = tf.concat([L, tempL[:, :(shape1-shape0)]], axis=1)
        mask = tf.concat([mask, tempM[:, :(shape1-shape0)]], axis=1)
      else:
        L = tf.concat([L, tempL], axis=1)
        mask = tf.concat([mask, tempM], axis=1)
    # the transpose condition
    if do_transpose:
      L = tf.transpose(L, perm=[1, 0])
      mask = tf.transpose(mask, perm=[1, 0])
    print('L:', L.shape, mask.shape)
    # L = tf.convert_to_tensor(L) # dtype=dtype
    # take the svd
    sigma, U, V = tf.linalg.svd(L)
    print(U, sigma, V)
    print_op = tf.print("tensors:", mask, L, sigma, U, V, 
                        output_stream=sys.stdout)  
    with tf.control_dependencies([print_op]):  
      tripled_tensor = L * 3
    sess2.run(tripled_tensor) 

[3, 2]
L: (6, 6) (6, 6)
Tensor("Svd:1", shape=(6, 6), dtype=float64) Tensor("Svd:0", shape=(6,), dtype=float64) Tensor("Svd:2", shape=(6, 6), dtype=float64)
tensors: [[1 1 1 1 0 0]
 [1 1 1 0 1 0]
 [1 1 1 0 0 1]
 [1 0 0 1 1 1]
 [0 1 0 1 1 1]
 [0 0 1 1 1 1]] [[3 -1 -1 -1 0 0]
 [-1 3 -1 0 -1 0]
 [-1 -1 3 0 0 -1]
 [-1 0 0 3 -1 -1]
 [0 -1 0 -1 3 -1]
 [0 0 -1 -1 -1 3]] [5.0000000000000062 5.0000000000000009 3.0000000000000018 3.0000000000000009 2.0000000000000018 6.5852386286146089e-17] [[0.32928587099560352 -0.47424060190582584 -0.21526894104448838 0.53571691811526545 -0.40824829046386291 0.40824829046386329]
 [-0.57534734425427 -0.048049628436564969 -0.35630998980268397 -0.45428683064793413 -0.408248290463863 0.40824829046386357]
 [0.24606147325866684 0.522290230342391 0.57157893084717226 -0.081430087467331588 -0.40824829046386329 0.40824829046386324]
 [-0.32928587099560352 0.47424060190582634 -0.21526894104448793 0.53571691811526512 0.408248290463863 0.40824829046386335]
 [0.5753473442542

In [ ]:
# Laplacian Small world topology
sess2 = tf.compat.v1.Session()  

def get_lap(R, topo):
    if topo=='chain':
      G = nx.grid_graph([R], periodic=True)
    elif topo=='grid':
      grid_dim = get_closest_int(R)
      print(grid_dim)
      G = nx.grid_graph(grid_dim, periodic=True)
    elif topo=='sw': # small world
      mean_degree = int(np.ceil(0.1*R))+1 # 10% of Total nodes
      print('R, k ', R, mean_degree)
      rewiring_prob = 0.5 # randomly choose 1/2 of the neiboring nodes from other nodes
      G = nx.connected_watts_strogatz_graph(R, mean_degree, rewiring_prob, seed=None)
    else:
      print('Laplacian topology not defined')
    A = nx.to_numpy_matrix(G)
    A = tf.convert_to_tensor(A)
    D = tf.reduce_sum(A, axis=-1)
    L = tf.linalg.diag(D)-A
    if R %2==0: # even
      mask = tf.linalg.diag(tf.ones(A.shape[-1], dtype=A.dtype)) + A
    else: # odd
      mask = A
    return L, mask

with sess2.as_default(): # initialize the reservoir weights
    shape = (6, 6)
    R = shape[-1]
    L, mask = get_lap(R, 'sw')
    print('L:', L.shape)
    # L = tf.convert_to_tensor(L) # dtype=dtype
    # take the svd
    eigval, eigvec = tf.linalg.eigh(L)
    print(eigval, eigvec)
    # print_op = tf.print("tensors:", mask, L, eigval, eigvec, 
    #                     output_stream=sys.stdout)  
    print_op = tf.print("Lap-SmallWorld:\n", L, 
                        output_stream=sys.stdout)  
    with tf.control_dependencies([print_op]):  
      tripled_tensor = L * 3
    sess2.run(tripled_tensor) 

R, k  6 2
L: (6, 6)
Tensor("SelfAdjointEigV2_1:0", shape=(6,), dtype=float64) Tensor("SelfAdjointEigV2_1:1", shape=(6, 6), dtype=float64)
Lap-SmallWorld:
 [[2 -1 0 0 0 -1]
 [-1 2 0 0 0 -1]
 [0 0 2 0 -1 -1]
 [0 0 0 1 -1 0]
 [0 0 -1 -1 2 0]
 [-1 -1 -1 0 0 3]]


In [ ]:
with sess2.as_default(): # initialize the reservoir weights
    shape = (6, 6)
    R = shape[-1]
    L, mask = get_lap(R, 'chain')
    print('L:', L.shape)
    # L = tf.convert_to_tensor(L) # dtype=dtype
    # take the svd
    eigval, eigvec = tf.linalg.eigh(L)
    print(eigval, eigvec)
    # print_op = tf.print("tensors:", mask, L, eigval, eigvec, 
    #                     output_stream=sys.stdout)  
    print_op = tf.print("Lap-Chain:\n", L, mask,
                        output_stream=sys.stdout)  
    with tf.control_dependencies([print_op]):  
      tripled_tensor = L * 3
    sess2.run(tripled_tensor) 

L: (6, 6)
Tensor("SelfAdjointEigV2_4:0", shape=(6,), dtype=float64) Tensor("SelfAdjointEigV2_4:1", shape=(6, 6), dtype=float64)
Lap-Chain:
 [[2 -1 0 0 0 -1]
 [-1 2 -1 0 0 0]
 [0 -1 2 -1 0 0]
 [0 0 -1 2 -1 0]
 [0 0 0 -1 2 -1]
 [-1 0 0 0 -1 2]] [[1 1 0 0 0 1]
 [1 1 1 0 0 0]
 [0 1 1 1 0 0]
 [0 0 1 1 1 0]
 [0 0 0 1 1 1]
 [1 0 0 0 1 1]]
